In [ ]:
!pip install langchain langchain-community langchain-text-splitters chromadb sentence-transformers pypdf transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.6 MB/s eta 0:00

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
#!pip install wget

In [ ]:

#import wget
#url= "https://www.aum.edu/wp-content/uploads/2025/05/AUM-Housing-Community-Standards.pdf"
#wget.download(url, "Housing_Community_Standards.pdf")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pdf_path = "/content/drive/My Drive/AUM-Housing-Community-Standards.pdf"

In [ ]:
import pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(pdf_path)
documents = loader.load()


In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)
chunks = splitter.split_documents(documents)


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


/tmp/ipython-input-4097373331.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})


In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
def generate_llm_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=350)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def rag_answer(question):
    # Retrieve relevant chunks
    docs = retriever.invoke(question)

    # Build hidden context
    context = "\n\n".join([d.page_content.strip() for d in docs])

    # Hidden prompt
    prompt = f"""
Using ONLY the information in the context, answer the user's question in one clear paragraph.
Do NOT repeat the question.
Do NOT mention the context.
Do NOT list steps or bullet points.
Do NOT include citations or formatting.
JUST answer the question directly in one short paragraph.

Context:
{context}

Question: {question}

Answer:
"""

    # Run the model
    raw_output = generate_llm_answer(prompt).strip()

    # Extract ONLY the answer (everything after "Answer:")
    if "Answer:" in raw_output:
        answer = raw_output.split("Answer:")[-1].strip()
    else:
        answer = raw_output.strip()

    return answer




In [ ]:
response = rag_answer("What is the quiet hours policy?")
print(response)



The quiet hours policy at AUM requires residents to keep noise levels to a minimum during specific times to ensure that all residents can sleep, study, relax, or host visitors without distracting noise from neighbors. Quiet hours are from 10:00 PM to 9:00 AM, Sunday through Thursday, and from 12:00 AM to 9:00 AM, Friday and Saturday. Residents are expected to comply with this policy both inside and outside the residence halls. If a resident's right to sleep or study during quiet hours is violated, they should politely ask those causing the disturbance to refrain. If the noise continues, they should inform the RA on duty. Failure to comply with the quiet hours policy may result in disciplinary action, including a $25 fine per occurrence.


In [ ]:
response = rag_answer("there is fire in my room what to do ?")
print(response)

In the event of a fire in your room, you must immediately activate the fire alarm to alert others. Then, call AUM Campus Police at 334-244-3424 to report the fire, providing your name, location, and any other pertinent information. Evacuate the building along with other residents and wait outside to inform the fire department and University officials about the fire. Do not re-enter the building until you have been notified by Housing and Residence Life staff or Campus Police that it is safe to do so.


In [ ]:
response = rag_answer("there is fire in my room what to do ?")
print(response)

In the event of a fire in your room, you must immediately activate the fire alarm to alert others. Then, call AUM Campus Police at 334-244-3424 to report the fire, providing your name, location, and any other pertinent information. Evacuate the building along with other residents and wait outside to inform the fire department and University officials about the fire. Do not re-enter the building until you have been notified by Housing and Residence Life staff or Campus Police that it is safe to do so.
